<a href="https://colab.research.google.com/github/jackashore/judgement/blob/develop/Ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import os
import re
import nltk
import pandas as pd
import random
from bs4 import BeautifulSoup
from string import punctuation
from pymorphy2 import MorphAnalyzer
from nltk.collocations import *
from nltk.corpus import stopwords
from nltk.util import everygrams
from collections import Counter, OrderedDict
from sklearn.feature_extraction.text import TfidfVectorizer

stops = set(stopwords.words('russian'))
punct = punctuation+'«»—…“”*№–).'
morph = MorphAnalyzer()

In [3]:
# nltk.download('stopwords')

In [4]:
court1 = pd.DataFrame()

In [22]:
# Функции скопированы из тетрадки про парсинг xml, со временем это превратится в прекрасного лебедя (импортируемый модуль)

meta_data = {'court': '', 'judge': '', 'prosecutor': '', 'secretary': '', 'accused': '', 'result': '', 'category': '', 
            'punishment_type': '', 'punishment_term': ''}

def parse_xml(path):   # парсим xml
    with open(path, 'r', encoding='utf-8')as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        
        meta_data['court'] = soup.court.string     # добавляем в метаданные то, что имеется в разметке
        meta_data['judge'] = soup.judge.string
        meta_data['result'] = soup.result.string
        meta_data['category'] = soup.category.string
        
        html = []
        for line in soup.body:
            line = line.replace('[', '')
            line = line.replace(']', '') 
            html.append(line)
        true_html = ' '.join(html)
                
        html_soup = BeautifulSoup(true_html, 'html.parser')
        return html_soup.get_text()
    
def get_parts(text):    # делим на части
    lines = [line for line in text.split('\n')]
    beg, end = 0, 0 
    for num, line in enumerate(lines):
        if re.search(r'у\s*с\s*т\s*а\s*н\s*о\s*в\s*и\s*л', line.lower()):
        #if 'установил' in line.lower() or 'у с т а н о в и л' in line.lower(): 
            beg = num + 1
        
        if re.search(r'п\s*р\s*и\s*г\s*о\s*в\s*о\s*р\s*и\s*л', line.lower()) \
        or re.search(r'п\s*о\s*с\s*т\s*а\s*н\s*о\s*в\s*и\s*л', line.lower()):
        #if re.search(r'[приговорил ]{10,}', line.lower()) or re.search(r'[постановил ]{10,}', line.lower()):
        #if 'приговорил' in line.lower() or 'п р и г о в о р и л' in line.lower() or 'постановил'\
         #in line.lower() or 'п о с т а н о в и л' in line.lower():
            end = num + 1

        if beg and end:
            beginning = [stroka.strip() for stroka in lines[:beg]]
            main_part = [stroka.strip() for stroka in lines[beg:end]]
            ending = [stroka.strip() for stroka in lines[end:]]
            return [' '.join(main_part), ' '.join(beginning), ' '.join(ending)]
        #else:
            #if '[CDATA[]]' in line.lower():
                
        
def clean(text):
    pattern1 = re.compile(r'(п|ч|ст)(\.|\s|\d)')             # обозначение статьи (пункт,часть,статья)
    pattern2 = re.compile(r'((У|Г)П?К|КоАП|ПДД)\sРФ')     # названия кодексов
    pattern3 = re.compile('\d')                            # цифры
    pattern4 = re.compile('(ДД.ММ.ГГ|дд.мм.гг)')           # даты

    text = re.sub(pattern1, ' ABBR ', text)
    text = re.sub(pattern2, ' DOCUMENT ', text)
    text = re.sub(pattern3, ' DIGIT ', text)
    text = re.sub(pattern4, ' DATE ', text)
    text = text.replace('\xad', '')
    text = text.replace('п р и г о в о р и л', '')
    text = text.replace('п о с т а н о в и л', '')
    text = text.replace('П Р И Г О В О Р И Л', '')
    text = text.replace('П О С Т А Н О В И Л', '')
    return text
        
def process_xml(path):
    try:
        return clean(get_parts(parse_xml(path))[0])
    except TypeError:
        return None

In [23]:
def get_texts(main_path):  # получаем тексты приговоров
    raw_texts = []
    for items in os.walk(main_path):
        for file in items[2]:
            # print(file)
            if file.split('.')[~0] == 'xml':
                file_path = os.path.join(items[0], file)
                raw_texts.append(process_xml(file_path))
    return raw_texts

In [7]:
# записываем тексты в датафрейм
court1['raw_main_parts'] = get_texts('data/leninskij-rajonnyj-sud-g-barnaula-altajskij-kraj-1102')

In [8]:
court1.head()

,raw_main_parts
0,"Котелевцев А.Г. совершил кражу, то есть тайно..."
1,Агулова М.А. совершила тайное хищение чужого и...
2,"Подсудимый Шакеров Р.А. совершил грабеж, то ес..."
3,+++ в период времени с DIGIT DIGIT часов D...
4,Лукьянченко А.В. совершил тайное хищение чужог...


* Прикрутить Верин токенизатор
* Подумать о лемматизаторе

In [60]:
def normalize(text):
    #trash = ['abbr', 'document', 'digit', 'date']
    words = [word.strip(punct) for word in text.lower().split()] 
    words = [word for word in words if word] # and word not in trash]
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

In [10]:
court1 = court1.dropna()

In [11]:
court1['norm_main_parts'] = court1['raw_main_parts'].apply(normalize)  # нормализуем

In [12]:
court1.head()

,raw_main_parts,norm_main_parts
0,"Котелевцев А.Г. совершил кражу, то есть тайно...","[котелевец, а.г, совершить, кража, то, есть, т..."
1,Агулова М.А. совершила тайное хищение чужого и...,"[агулов, м.а, совершить, тайный, хищение, чужо..."
2,"Подсудимый Шакеров Р.А. совершил грабеж, то ес...","[подсудимый, шакер, р.а, совершить, грабёж, то..."
3,+++ в период времени с DIGIT DIGIT часов D...,"[в, период, время, с, часы, минута, до, часы, ..."
4,Лукьянченко А.В. совершил тайное хищение чужог...,"[лукьянченко, а.в, совершить, тайный, хищение,..."


In [15]:
def get_ngrams(court):   # выделяем 1000 или 5000 самых частотных энграмм
    all_texts = []
    colloc_1000 = [] 
    for text in court.norm_main_parts:    # записываем все тексты в одно место
        all_texts.extend(text)
    for col in Counter(list(everygrams(all_texts, min_len=3, max_len=20))).most_common(5000):
        colloc_1000.append(' '.join(col[0]))
    return colloc_1000

In [16]:
gold_std = get_ngrams(court1)    # 5000 частотных энграмм большого суда

In [18]:
print(*(gold_std[:20]), sep='\n') # , get_ngrams(court2)[:20])

в судебный заседание
кодекс российский федерация
в соответствие с
без цель сбыт
степень общественный опасность
сбыт наркотический средство
в связь с
характер и степень
и степень общественный
и степень общественный опасность
хищение чужое имущество
характер и степень общественный
характер и степень общественный опасность
явка с повинный
уголовный кодекс российский
уголовный кодекс российский федерация
по адрес адрес
цель сбыт наркотический
цель сбыт наркотический средство
без цель сбыт наркотический


In [19]:
random.shuffle(os.listdir('data'))     # взяли 10 рандомных судов
ten_courts = os.listdir('data')[:10]

Дальше нужно снова выделить для этих 10 судов их частотные энграммы и по очереди пересечь с `gold_std`

In [20]:
ten_courts

['krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986',
 'kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927',
 'kulundinskij-rajonnyj-sud-altajskij-kraj-1040',
 'primorskij-rajonnyj-sud-arxangelskaya-oblast-1632',
 'korochanskij-rajonnyj-sud-belgorodskaya-oblast-941',
 'verxnepyshminskij-gorodskoj-sud-sverdlovskaya-oblast-400',
 'saratovskij-oblastnoj-sud-saratovskaya-oblast-2403',
 'ketchenerovskij-rajonnyj-sud-respublika-kalmykiya-834',
 'bogdanovichskij-gorodskoj-sud-sverdlovskaya-oblast-310',
 'severouralskij-gorodskoj-sud-sverdlovskaya-oblast-1768']

In [21]:
court_dict = {court:'court_'+str(i) for i, court in enumerate(ten_courts, 2)}
court_dict

{'krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986': 'court_2',
 'kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927': 'court_3',
 'kulundinskij-rajonnyj-sud-altajskij-kraj-1040': 'court_4',
 'primorskij-rajonnyj-sud-arxangelskaya-oblast-1632': 'court_5',
 'korochanskij-rajonnyj-sud-belgorodskaya-oblast-941': 'court_6',
 'verxnepyshminskij-gorodskoj-sud-sverdlovskaya-oblast-400': 'court_7',
 'saratovskij-oblastnoj-sud-saratovskaya-oblast-2403': 'court_8',
 'ketchenerovskij-rajonnyj-sud-respublika-kalmykiya-834': 'court_9',
 'bogdanovichskij-gorodskoj-sud-sverdlovskaya-oblast-310': 'court_10',
 'severouralskij-gorodskoj-sud-sverdlovskaya-oblast-1768': 'court_11'}

In [22]:
def get_all_done(dictionary):
    d_ngrams = {}
    for court, df in dictionary.items():
        print(court)
        df = pd.DataFrame()
        df['raw_main_parts'] = get_texts(os.path.join('data', court))
        df = df.dropna()
        print('got texts for %s' % court)
        print(df.shape)
        df['norm_main_parts'] = df['raw_main_parts'].apply(normalize)
        print('%s normalized' % court)
        d_ngrams[court] = get_ngrams(df)
        print('got ngrams for %s' % court)
    return d_ngrams

In [23]:
courts_ngrams = get_all_done(court_dict)

krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986
got texts for krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986
(274, 1)
krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986 normalized
got ngrams for krasnozerskij-rajonnyj-sud-novosibirskaya-oblast-986
kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927
got texts for kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927
(364, 1)
kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927 normalized
got ngrams for kondinskij-rajonnyj-sud-xanty-mansijskij-avtonomnyj-okrug-927
kulundinskij-rajonnyj-sud-altajskij-kraj-1040
got texts for kulundinskij-rajonnyj-sud-altajskij-kraj-1040
(349, 1)
kulundinskij-rajonnyj-sud-altajskij-kraj-1040 normalized
got ngrams for kulundinskij-rajonnyj-sud-altajskij-kraj-1040
primorskij-rajonnyj-sud-arxangelskaya-oblast-1632
got texts for primorskij-rajonnyj-sud-arxangelskaya-oblast-1632
(445, 1)
primorskij-rajonnyj-sud-arxangelskaya-oblast-1632 normalized
got ngrams for primorskij

In [24]:
for ngrams in courts_ngrams.values():
    final_ngrams = []
    final_ngrams.extend(set(gold_std)&set(ngrams))
final_ngrams=set(final_ngrams)
print(final_ngrams)

{'по уголовный кодекс', 'уголовный кодекс российский федерация в', 'не находить основание для', 'не находить основание', 'психотропный вещество и', 'в тот число', 'ходатайство о постановление приговор без проведение судебный', 'и личность виновный', 'есть открытый хищение чужое имущество', 'следующий обстоятельство год около', 'цель сбыт наркотический средство в особо', 'применение насилие не', 'магазин дать изъять', 'год в период', 'крупный размер при', 'средство в особо крупный', 'есть тайный хищение чужое имущество совершенный', 'наказание суд учитывать', 'обстоятельство год около', 'влияние назначить наказание на исправление осудить и на условие жизнь', 'квартира по адрес адрес', 'грабёж то есть открытый', 'на исправление осудить и на условие жизнь', 'осознать характер и', 'россия по год', 'в установленный закон', 'по уголовный дело', 'в состояние алкогольный опьянение', 'о постановление приговор', 'без цель сбыт наркотический средство в особо', 'с часы до', 'наказание суд учитыват

In [25]:
len(final_ngrams)

495

In [26]:
with open('juridisms.txt', 'w', encoding='utf-8') as f:
    for item in final_ngrams:
        f.write(item+'\n')

# Тестим деление на параграфы

In [204]:
def find_paragraphs(main):
    """
    main -- основная часть дела, задается списком предложений 
    """
    f, w, p, m = 0, 0, 0, 0
    fab, witness, proof, meditation = [], [], [], []
    for i, par in enumerate(main):
        if ('подтверждается' in par.lower() and 'доказательствами' in par.lower()) or (
            'свидетель ' in par.lower()) or ('показал ' in par.lower()):
            f = i
            fab = main[:f]
            fab = ' '.join([x for x in fab])
            if 'квалифицирован' in main[i+1].lower() or 'суд' in main[i+1].lower():
                #m = i
                meditation = main[f:]
                meditation = ' '.join([x for x in meditation])
        if ('суд убедился' in main[i].lower()) or ('судья убедилась' in main[i].lower()):
            f = i
            fab = main[:f]
            fab = ' '.join([x for x in fab])
            meditation = main[f:]
            meditation = ' '.join([x for x in meditation])
        if 'подтверждается' in par.lower() and 'материалами дела' in par.lower() or ('согласно' in par.lower()) or (
            'по заключению' in par.lower()):
            w = i
            if w != 0:
                witness = main[f:w]
                witness = ' '.join([x for x in witness])
        if ('перечисленные' in par.lower() and 'выше' in par.lower() and 'доказательства' in par.lower()) or ('оценив' in par.lower()):
            p = i
            if w != 0 :
                proof = main[w:p]
                proof = ' '.join([x for x in proof])
                meditation = main[p:len(main)]
                meditation = ' '.join([x for x in meditation])
    return {'fabula': fab, 'witness': witness, 'proof': proof, 'meditation': meditation}

In [ ]:
{'fabula': '13 января 2009 года Куулар, с целью временно уклониться от военной службы, желая побыть с родственниками, в установленный срок без уважительных причин не явился на службу в войсковую часть № и убыл к месту жительства в <адрес>, где проживал, проводя время по своему усмотрению. 16 апреля 2011 года Куулар обнаружен представителем военного следственного отдела по Абаканскому гарнизону по месту жительства. Органами предварительного следствия Куулар, кроме того, обвинялся в совершении преступления, предусмотренного ч. 3 ст. 337 УК РФ, однако государственный обвинитель в судебном заседании отказался от этого обвинения в соответствии с ч. 7 ст. 246 УПК РФ, на основании п. 2 ч. 1 ст. 24 УК РФ, а по этому постановлением судьи от 23 ноября 2011 года в данной части уголовное дело прекращено. Подсудимый Куулар виновным себя в неявке в срок без уважительных причин на службу признал полностью, об обстоятельствах содеянного дал показания, соответствующие изложенному выше и пояснил, что не прибыл на службу в войсковую часть № так как хотел побыть с родственниками.',
 'witness': 'Виновность Куулара подтверждается следующими доказательствами. Из показаний свидетеля З. командира батареи, усматривается, что Куулар 13 января 2011 года не явился на службу в войсковую часть № без уважительных причин и более в части не появлялся. Согласно показаниям свидетеля М. 16 апреля 2011 года он обнаружил Куулара в селе <адрес> и доставил его в военный следственный отдел по Абаканскому гарнизону.',
 'proof': 'Из заключения экспертов-психиатров усматривается, что Куулар каким-либо хроническим психическим расстройством не страдал и не страдает, в стационарном обследовании не нуждается, является психически здоровым человеком. По состоянию психического здоровья он годен к военной службе. Оценив данное заключение экспертов-психиатров в совокупности с другими исследованными в суде доказательствами, суд признает Куулара вменяемым и ответственным за содеянное. Согласно заключению военно-врачебной комиссии, Куулар годен к военной службе.',
 'meditation': 'Оценив изложенные доказательства в совокупности, суд признает их достоверными и достаточными, а виновность подсудимого Куулара в содеянном доказанной. Действия Куулара по уклонению с 13 января 2009 года по 16 апреля 2011 года суд квалифицирует по ч. 4 ст. 337 УК РФ, поскольку он, являясь военнослужащим, проходящим военную службу по контракту, не явился в срок без уважительных причин на службу в войсковую часть № продолжительностью свыше одного месяца. Обстоятельством, смягчающим наказание Куулару, суд признает его раскаяние в содеянном. При назначении наказания подсудимому суд принимает во внимание, что он до поступления на военную службу, в быту и по службе характеризуется положительно. Так же суд учитывает, что Куулару постановлением судьи <адрес> районного суда от (дата) отменено условное осуждение, назначенное приговором <адрес> районного суда <адрес> от (дата) по ч. 1 ст. 166 УК РФ сроком на 3 года лишения свободы и данное наказание назначено к исполнению с учетом личности виновного с его отбытием в исправительной колонии общего режима. Ввиду того, что Куулар в ходе длительного уклонения от военной службы (более двух лет) скрывал свою принадлежность к Вооруженным Силам РФ, при этом совершил умышленное преступление средней тяжести и был за это осужден, то суд полагает необходимым назначить ему наказание в виде лишения свободы с его отбыванием в исправительной колонии общего режима. Поскольку подсудимый Куулар ходатайствовал о рассмотрении дела в особом порядке, но судом ему в данном ходатайстве было отказано, суд считает необходимым процессуальные издержки, связанные с оплатой услуг адвоката, возместить за счет средств федерального бюджета. На основании изложенного и руководствуясь ст.ст. 307, 308 и 309 УПК РФ, военный суд ПРИГОВОРИЛ:'}

In [205]:
find_paragraphs(court_test['raw_main_parts'][0].)

{'fabula': [], 'witness': [], 'proof': [], 'meditation': []}

# Тестим поиск юридизмов

In [122]:
def find_sublists(haystack, needle):
    """
    Выполняет поиск списка в списке (или строку в подстроке) по алгоритму Бойера - Мура - Хорспула
    :param haystack: Где ищем
    :param needle: Что ищем
    :return: Позиция первого элемента needle в haystack или -1, если не было найдено.
    """
    h = len(haystack)
    n = len(needle)

    # Таблица смещений вида: "элемент" -> "величина сдвига"
    # Величина сдвига определяет, на сколько позиций влево можно сдвинуть шаблон, чтобы не пропустить ничего важного.
    skip = {needle[i]: n - i - 1 for i in range(n - 1)}

    # Шаблон всегда сравниваем с последнего символа к первому
    i = n - 1

    # Главный цикл по большому массиву
    found = []
    while i < h:
        # Цикл сравнения шаблона
        for j in range(n):
            # Сравнение конкретного элемента массива
            if haystack[i - j] != needle[-j - 1]:
                # Если элементы не равны, то выполняем сдвиг шаблона согласно составленной ранее таблице.
                # Причем если элемента в таблице нет, то сдвигаем на всю величину шаблона
                i += skip.get(haystack[i], n)
                break
        # Если все сравнения прошли успешно, то возвращаем индекс элемента
        else:
            found.append(i - n + 1)
            i += skip.get(haystack[i], n)
    return found

In [29]:
court_test = pd.DataFrame()

In [30]:
court_test['raw_main_parts'] = get_texts('data/agryzskij-rajonnyj-sud-respublika-tatarstan-132')

In [141]:
court_test.head()

,raw_main_parts,norm_main_parts,raw_splitted
0,DATE ГГ в период с DIGIT часов до DIGIT ...,"[date, год, в, период, с, digit, часы, до, dig...","[DATE, ГГ, в, период, с, DIGIT, часов, до, DIG..."
1,Подсудимый совершил преступление при следующих...,"[подсудимый, совершить, преступление, при, сле...","[Подсудимый, совершил, преступление, при, след..."
2,Подсудимый совершил преступление при следующих...,"[подсудимый, совершить, преступление, при, сле...","[Подсудимый, совершил, преступление, при, след..."
3,DATE ГГ в период с DIGIT DIGIT . DIGIT DIG...,"[date, год, в, период, с, digit, digit, digit,...","[DATE, ГГ, в, период, с, DIGIT, DIGIT, DIGIT, ..."
4,Подсудимый совершил преступление при следующих...,"[подсудимый, совершить, преступление, при, сле...","[Подсудимый, совершил, преступление, при, след..."


In [32]:
court_test.shape

(124, 1)

In [33]:
court_test = court_test.dropna()

In [34]:
court_test['norm_main_parts'] = court_test['raw_main_parts'].apply(normalize)

In [172]:
len(court_test['norm_main_parts'][1])

473

In [174]:
splitted = [item.strip() for item in court_test['raw_main_parts'][1].split()] # ?? if item not in punct]
len(splitted)

489

In [189]:
cool_text = 'Привет я купил большого слона! но суд постановил что мне так делать было нельзя'

bad_lemmas = ['суд постановить', 'большой слон']

In [200]:
cool_text1 = 'Привет я купил большого слона! но суд постановил что мне так делать было нельзя'

string = ''

for word in m.split(cool_text1):
    for i in bad_lemmas:
        lemma = m.morph(word)[0].normalized
        string = string + lemma + ' '
        if i in string:
            string = string.replace(i, 'TAG')

In [201]:
print(string)

привет привет я я купить купить большой TAG слон ! ! но но суд TAG постановить что что я я так так делать делать быть быть нельзя нельзя 


In [203]:
from yargy.tokenizer import MorphTokenizer

m = MorphTokenizer()
cool_text2 = 'суд постановил суд суд суд принял'

for i in bad_lemmas:
    for word in m.split(cool_text2):
        lemma = m.morph(word)[0].normalized
        if lemma in i.split():
            cool_text2 = cool_text2.replace(word, 'TAG')
print(cool_text2)

TAG TAG TAG TAG TAG принял


In [178]:
from nltk import word_tokenize

word_tokenize(court_test['raw_main_parts'][1])

['Подсудимый',
 'совершил',
 'преступление',
 'при',
 'следующих',
 'обстоятельствах',
 '.',
 'DATE',
 'ГГ',
 'около',
 'DIGIT',
 'DIGIT',
 'часов',
 'DIGIT',
 'DIGIT',
 'минут',
 ',',
 'Коробейников',
 'А.С.',
 ',',
 'находясь',
 'в',
 'автомобиле',
 '<',
 'данные',
 'изъяты',
 '>',
 ',',
 'принадлежащего',
 'ФИО',
 'DIGIT',
 ',',
 'расположенном',
 'во',
 'дворе',
 '<',
 'адрес',
 '>',
 ',',
 'употреблял',
 'спиртные',
 'напитки',
 'совместно',
 'с',
 'ФИО',
 'DIGIT',
 '.',
 'После',
 'распития',
 'спиртных',
 'напитков',
 'ФИО',
 'DIGIT',
 'опьянен',
 'ушел',
 'домой',
 'спать',
 ',',
 'оставив',
 'ключи',
 'в',
 'замке',
 'зажигания',
 'автомобиля',
 '.',
 'Коробейников',
 'А.С',
 '.',
 'находясь',
 'в',
 'вышеуказанном',
 'месте',
 ',',
 'в',
 'указанное',
 'время',
 ',',
 'увидел',
 'ключи',
 'в',
 'замке',
 'зажигания',
 'автомобиля',
 'и',
 'у',
 'него',
 'возник',
 'преступный',
 'умысел',
 ',',
 'направленный',
 'на',
 'неправомерное',
 'завладение',
 'без',
 'цели',
 'хищени

In [179]:
len(word_tokenize(court_test['raw_main_parts'][1]))

560

In [175]:
splitted

['Подсудимый',
 'совершил',
 'преступление',
 'при',
 'следующих',
 'обстоятельствах.',
 'DATE',
 'ГГ',
 'около',
 'DIGIT',
 'DIGIT',
 'часов',
 'DIGIT',
 'DIGIT',
 'минут,',
 'Коробейников',
 'А.С.,',
 'находясь',
 'в',
 'автомобиле',
 '<данные',
 'изъяты>,',
 'принадлежащего',
 'ФИО',
 'DIGIT',
 ',',
 'расположенном',
 'во',
 'дворе',
 '<адрес>,',
 'употреблял',
 'спиртные',
 'напитки',
 'совместно',
 'с',
 'ФИО',
 'DIGIT',
 '.',
 'После',
 'распития',
 'спиртных',
 'напитков',
 'ФИО',
 'DIGIT',
 'опьянен',
 'ушел',
 'домой',
 'спать,',
 'оставив',
 'ключи',
 'в',
 'замке',
 'зажигания',
 'автомобиля.',
 'Коробейников',
 'А.С.',
 'находясь',
 'в',
 'вышеуказанном',
 'месте,',
 'в',
 'указанное',
 'время,',
 'увидел',
 'ключи',
 'в',
 'замке',
 'зажигания',
 'автомобиля',
 'и',
 'у',
 'него',
 'возник',
 'преступный',
 'умысел,',
 'направленный',
 'на',
 'неправомерное',
 'завладение',
 'без',
 'цели',
 'хищения',
 'вышеуказанным',
 'автомобилем.',
 'Коробейников',
 'А.С.',
 'с',
 'це

In [136]:
def split_text(text):
    splitted = [item.strip(punct) for item in text.split() if item not in punct]
    return [item for item in splitted if item]

In [140]:
court_test['raw_splitted'] = court_test['raw_main_parts'].apply(split_text)

In [173]:
del_jur(splitted, court_test['norm_main_parts'][1], all_juridisms)

по уголовный кодекс
уголовный кодекс российский федерация в
не находить основание для
не находить основание
психотропный вещество и
в тот число
ходатайство о постановление приговор без проведение судебный
и личность виновный
есть открытый хищение чужое имущество
следующий обстоятельство год около
цель сбыт наркотический средство в особо
применение насилие не
магазин дать изъять
год в период
крупный размер при
средство в особо крупный
есть тайный хищение чужое имущество совершенный
наказание суд учитывать
[368]
обстоятельство год около
влияние назначить наказание на исправление осудить и на условие жизнь
квартира по адрес адрес
грабёж то есть открытый
на исправление осудить и на условие жизнь
осознать характер и
россия по год
в установленный закон
по уголовный дело
в состояние алкогольный опьянение
о постановление приговор
без цель сбыт наркотический средство в особо
с часы до
наказание суд учитывать характер и степень общественный опасность
цель сбыт наркотический средство в особо круп

'Подсудимый совершил преступление JUR JUR JUR DATE ГГ около DIGIT DIGIT часов DIGIT DIGIT минут, Коробейников А.С., находясь в автомобиле <данные изъяты>, принадлежащего ФИО DIGIT расположенном во дворе <адрес>, употреблял спиртные напитки совместно с ФИО DIGIT После распития спиртных напитков ФИО DIGIT опьянен ушел домой спать, оставив ключи в замке зажигания автомобиля. Коробейников А.С. находясь в вышеуказанном месте, в указанное время, увидел ключи в замке зажигания автомобиля и у него возник преступный умысел, направленный на неправомерное завладение без цели хищения вышеуказанным автомобилем. Коробейников А.С. с целью реализации возникшего преступного умысла направленного на неправомерное завладение данной автомашиной, без цели хищения, зная, что он не имеет права на управление и распоряжение данным транспортным средством, осознавая JUR JUR JUR JUR предвидя возможность наступления общественно опасных последствий и желая их наступления, действуя умышленно, с целью кататься по горо

In [157]:
def del_jur(raw_text, norm_text, juridisms):
    """
    :param raw_text: list
    :param norm_text: list
    :param juridisms: list
    """
    for juridism in juridisms:
        print(juridism)
        found = find_sublists(norm_text, juridism.split())
        if found:
            print(found)
            for item in found:
                raw_text[item:item+len(juridism.split())] = ['JUR']*len(juridism.split())
    return ' '.join(raw_text)

In [147]:
with open('juridisms.txt', 'r') as f:
    all_juridisms = [line.strip() for line in f.readlines()]

In [159]:
text_wo_jurs = del_jur(court_test['raw_splitted'][0], court_test['norm_main_parts'][0], all_juridisms)

по уголовный кодекс
уголовный кодекс российский федерация в
не находить основание для
не находить основание
психотропный вещество и
в тот число
ходатайство о постановление приговор без проведение судебный
и личность виновный
есть открытый хищение чужое имущество
следующий обстоятельство год около
цель сбыт наркотический средство в особо
применение насилие не
магазин дать изъять
[881, 1646, 2000, 2050]
год в период
[1, 2274]
крупный размер при
средство в особо крупный
есть тайный хищение чужое имущество совершенный
[2478]
наказание суд учитывать
обстоятельство год около
влияние назначить наказание на исправление осудить и на условие жизнь
квартира по адрес адрес
грабёж то есть открытый
на исправление осудить и на условие жизнь
осознать характер и
россия по год
в установленный закон
по уголовный дело
в состояние алкогольный опьянение
[74, 2640, 2662]
о постановление приговор
без цель сбыт наркотический средство в особо
с часы до
наказание суд учитывать характер и степень общественный опа

[2220]
уголовный дело в
с незаконный проникновение в
дать о личность
[2593]
уголовно-процессуальный кодекс российский федерация суд
консультация с защитник
период время с
наказание в вид лишение
[2680]
тайный хищение чужое имущество
[2479]
наказание суд учитывать характер
и общественный опасность свой действие
с часы до часы
приговор без проведение судебный
на условие жизнь он
насилие не опасный для жизнь и здоровье
то есть тайный хищение чужое имущество совершенный с
[2477]
основание для применение
насилие не опасный для жизнь
жизнь и здоровье
соответствие с уголовный кодекс российский
исправление осудить и на условие жизнь
хранение без цель сбыт наркотический
приходить к вывод о
хищение чужое имущество совершенный с
[2480]
по вышеуказанный адрес
с уголовный кодекс
постановление приговор без проведение судебный
учитывать характер и степень
[2585]
грабёж то есть открытый хищение чужое имущество
то есть открытый хищение чужое имущество
опасный для жизнь и
осудить и на условие жизнь он с

## Здесь начинается ненужное

In [ ]:
len(set(get_ngrams(court1)) & set(get_ngrams(court2)))

In [ ]:
def tf_idf(court):    # че пацаны тф идф 
    tfidf = TfidfVectorizer(ngram_range=(3,10), min_df=10)
    
    court['tokens_str'] = [' '.join(i) for i in court.norm_main_parts]
    
    texts_vectors = tfidf.fit_transform(court['tokens_str'])
    
    id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
    
    court['tf-idf'] = pd.Series()
    court['tf-idf'] = court['tf-idf'].astype(object)
    
    some_array = []
    for i in range(len(court.tokens_str)):
        some_dict = {}
        for num, s in enumerate(texts_vectors.toarray()[i]):
            some_dict.setdefault((id2word[num]), s)
        sorted_dict = OrderedDict(sorted(some_dict.items(), key=lambda t: t[1])[::-1]) # сортировка значений по убыванию
        minim_array = [item[0] for item in sorted_dict.items() if item[1]!=0]
        court.iat[i,3] = minim_array

In [ ]:
tf_idf(court1)

In [ ]:
court1.head()

In [ ]:
df['jur'] = pd.Series()
df['jur'] = df['jur'].astype(object)
for num, item in enumerate(df['tf-idf']):
    df.iloc[num, 4] = set(colloc_1000) & set(item)

In [ ]:
smth = ' '.join(df['norm_main_parts'][0])
for j in df['jur'][0]:
    smth = smth.replace(j, 'JUR')

In [ ]:
for i in df['jur']:
    print(i)

In [ ]:
def del_jur(text):
    norm_text = ' '.join(normalize(text))
    for i in jur:
        norm_text = norm_text.replace(i, 'JUR')

In [ ]:
from pymystem3 import Mystem

m = Mystem()
analysis = m.analyze(df['raw_main_parts'][0])
lemmas = ''
txt = ''
for item in analysis:
    if len(item) > 1:
        for i in item['analysis']:
            lemmas += i['lex'] + ' '
            txt += item['text'] + ' '

In [ ]:
d = {}
for l, t in zip(lemmas.split(), txt.split()):
    d[t] = l

In [ ]:
for j in df['jur'][0]:
    juridism = j.split()
    print(juridism)
    if all(juridism) in lemmas.split():
        print('ok')
        print(lemmas.index(juridism[0]), lemmas.index(juridism[1]))

In [ ]:
lemmas.split()

In [ ]:
l = ['a','b','c','d']
if all(['a','b']) in l:
    print('ok')
    print(l.index('a'), l.index('b'))